In [ ]:
import urllib.request
import datetime
import time
import os

if not os.path.isdir('df'):
    os.mkdir('df')
for i in range(1,28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    print("-")
    wp = urllib.request.urlopen(url)
    print("--")
    text = wp.read()
    print("---")
    now = datetime.datetime.now()
    print("+")
    date_and_time = now.strftime("%d-%m-%Y__%H-%M-%S")
    print("++")

    with open(f'df/id_{str(i)}_{date_and_time}.csv', 'wb') as f:
        f.write(text)
    print("saved file")

print("VHI is downloaded...")


In [30]:
import os
import pandas as pd

def read_csv_files_to_df(directory):
    data_frames = []  # список для збереження DataFrame з кожного файлу

    # Проходимо по всіх файлах у вказаній директорії
    for filename in os.listdir(directory):
        reg = filename.split("_")[1]
        file_path = os.path.join(directory, filename)

        try:
            df = pd.read_csv(file_path, index_col=False, header=1)
            df.columns = [col.strip().lower().replace("<br>", "") for col in df.columns]
            df = df.replace(to_replace=r'<.*?>', value='', regex=True)
            df["oblast"] = reg
            data_frames.append(df)

        except Exception as e:
            print(f"Не вдалося зчитати файл {file_path}: {e}")

    # Об'єднуємо всі датафрейми в один
    if data_frames:
        combined_df = pd.concat(data_frames, ignore_index=True)

    else:
        combined_df = pd.DataFrame()

    return combined_df

df = read_csv_files_to_df("df")




In [33]:
# Замінити -1 значенням з попереднього рядка
while (df[['vci', 'tci', 'vhi']] == -1).any().any():
    df = df.mask(df == -1, df.shift())

#видалити всі рядки, де принаймні в одному зі стовпців зазначених нижче немає даних
df = df.dropna(subset=['vci', 'tci', 'vhi'])

,year,week,smn,smt,vci,tci,vhi,oblast
2391,1984,51.0,0.079,267.96,39.61,31.45,35.53,11
2392,1984,52.0,0.079,267.96,39.61,31.45,35.53,11
2393,1985,1.0,0.079,267.96,39.61,31.45,35.53,11
2394,1985,2.0,0.079,267.96,39.61,31.45,35.53,11
2395,1985,3.0,0.079,267.96,39.61,31.45,35.53,11
2396,1985,4.0,0.079,267.96,39.61,31.45,35.53,11
2397,1985,5.0,0.079,267.96,39.61,31.45,35.53,11
2398,1985,6.0,0.079,267.96,39.61,31.45,35.53,11
2399,1985,7.0,0.079,267.96,39.61,31.45,35.53,11
2400,1985,8.0,0.079,267.96,39.61,31.45,35.53,11


In [34]:
df.to_csv('df/combined_df.csv', index=False)
df = pd.read_csv('df/combined_df.csv', index_col=False)


In [10]:
obl_dict = {
    1: "Черкаська",
    2: "Чернігівська",
    3: "Чернівецька",
    4: "АР Крим",
    5: "Дніпропетровська",
    6: "Донецька",
    7: "Івано-Франківська",
    8: "Харківська",
    9: "Херсонська",
    10: "Хмельницька",
    11: "Київська",
    12: "м. Київ",
    13: "Кіровоградська",
    14: "Луганська",
    15: "Львівська",
    16: "Миколаївська",
    17: "Одеська",
    18: "Полтавська",
    19: "Рівненська",
    20: "м. Севастополь",
    21: "Сумська",
    22: "Тернопільська",
    23: "Закарпатська",
    24: "Вінницька",
    25: "Волинська",
    26: "Запорізька",
    27: "Житомирська"
}

df["oblast"] = df["oblast"].astype(int).map(obl_dict)


In [5]:
#перетворити ствопчик років з рядка на ціле число
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

print(df[(df["year"]==2018)][:10])


      year  week    smn     smt    vci    tci    vhi       oblast
1872  2018   1.0  0.071  260.87  63.46  35.69  49.57  Хмельницька
1873  2018   2.0  0.070  259.63  64.12  40.79  52.46  Хмельницька
1874  2018   3.0  0.071  259.85  64.14  45.75  54.95  Хмельницька
1875  2018   4.0  0.071  258.79  62.36  46.69  54.53  Хмельницька
1876  2018   5.0  0.065  257.71  53.36  51.54  52.45  Хмельницька
1877  2018   6.0  0.063  259.61  42.31  57.21  49.76  Хмельницька
1878  2018   7.0  0.064  261.47  38.31  55.28  46.80  Хмельницька
1879  2018   8.0  0.068  262.87  38.86  56.20  47.53  Хмельницька
1880  2018   9.0  0.077  264.33  40.67  57.98  49.32  Хмельницька
1881  2018  10.0  0.088  266.63  43.68  55.04  49.36  Хмельницька


In [6]:
import statistics

def vhi_obl_year(obl, year):
    res = df[(df["oblast"] == obl) & (df["year"] == year)]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {year} рік склав \n{res_vhi}")

    print("\n\tMIN:", min(res_vhi))
    print("\tMAX:", max(res_vhi))

    print("\n\tAVG:", statistics.mean(res_vhi))
    print("\tMED:", statistics.median(res_vhi))

vhi_obl_year("Запорізька", 2003)


vhi для Запорізька області на 2003 рік склав 
[62.06, 58.22, 55.64, 53.87, 51.0, 48.04, 45.78, 44.68, 44.68, 43.78, 44.92, 45.6, 42.9, 37.98, 35.01, 32.6, 29.24, 24.95, 19.46, 16.56, 15.92, 16.23, 19.7, 27.95, 38.84, 49.68, 59.3, 68.31, 76.1, 81.03, 84.13, 85.29, 84.29, 83.4, 81.65, 76.92, 70.18, 64.69, 64.94, 61.27, 52.65, 51.06, 48.12, 43.34, 39.43, 38.65, 40.21, 41.47, 44.48, 49.2, 54.47, 59.63]

	MIN: 15.92
	MAX: 85.29

	AVG: 50.18269230769231
	MED: 48.08


In [ ]:
def vhi_obls_years(obl, min_year, max_year):
    res = df[(df["oblast"].isin(obl)) & (df["year"].between(min_year, max_year+1))]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {min_year}-{max_year} має {len(res_vhi)} значень")

    print("\n\tMIN:", min(res_vhi))
    print("\tMAX:", max(res_vhi))

    print("\n\tAVG:", statistics.mean(res_vhi))
    print("\tMED:", statistics.median(res_vhi))

vhi_obls_years(["Запорізька", "Волинська", "Херсонська"], 1990, 2021)


In [ ]:
def drought_years(percent=0):
    draught_years = []
    oblasti = df["oblast"].unique()
    min_obl_num = int(percent * len(oblasti) / 100)

    for year in df["year"].unique():
        draughts_in = 0
        for obl in oblasti:
            res = df[(df["oblast"] == obl) & (df["year"] == year)]
            res_vhi = list(res["vhi"])
            if len(res_vhi) != 0:
                if statistics.mean(res_vhi) < 35:
                    draughts_in += 1

        if draughts_in > min_obl_num:
            draught_years.append(int(year))

    print(f"Було {len(draught_years)} посушливих років")
    print(draught_years)

drought_years(20)
